<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Custom Machine Learning engine (configuration & validation)

This notebook shows how to configure OpenScale to work with custom model serving engine using python sdk.

### Contents

- [Setup](#setup)
- [OpenScale data mart configuration](#openscale)
- [Subscription creation and testing](#performance)
- [Quality monitor configuration and testing](#quality)
- [Fairness monitoring and explanations configuration and testing](#fairness)

# Setup <a name="setup"></a>

## Custom machine learning engine example

The sample machine learning engine code (python flask) and deployment instructions can be found [here](https://github.com/pmservice/ai-openscale-tutorials/tree/master/applications/custom-ml-engine-bluemix).

Follow up intructions and deploy your own instance of the custom engine application.

**NOTE:** CUSTOM machine learning engine must follow this [API specification](https://aiopenscale-custom-deployement-spec.mybluemix.net/) to be supported.

## Packages and credentials

In [1]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

You should consider upgrading via the 'pip install --upgrade pip' command.


### ACTION: Restart the kernel

Import and initiate.

In [2]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
import pandas as pd

#### ACTION: Get OpenScale `instance_guid` and `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

How to get your OpenScale instance GUID

- if your resource group is different than `default` switch to resource group containing OpenScale instance
```
bx target -g <myResourceGroup>
```
- get details of the instance
```
bx resource service-instance 'AI-OpenScale-instance_name'
```

#### Let's define some constants required to set up data mart:

- AIOS_CREDENTIALS
- POSTGRES_CREDENTIALS
- SCHEMA_NAME

In [3]:
AIOS_CREDENTIALS = {
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "instance_guid": "***",
  "apikey": "***"
}

In [5]:
POSTGRES_CREDENTIALS = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [7]:
SCHEMA_NAME = 'custom_ml_engine'

Create schema for data mart.

In [8]:
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

In [9]:
client = APIClient(AIOS_CREDENTIALS)

In [10]:
client.version

'2.1.7'

## Training data

#### Get training data set from git

In [12]:
!rm credit_risk_training.csv 
!wget https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spss/credit-risk/data/credit_risk_training.csv

rm: cannot remove ‘credit_risk_training.csv’: No such file or directory
--2019-04-18 08:04:17--  https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spss/credit-risk/data/credit_risk_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694222 (678K) [text/plain]
Saving to: ‘credit_risk_training.csv’

100%[======================================>] 694,222     --.-K/s   in 0.02s   

2019-04-18 08:04:17 (27.2 MB/s) - ‘credit_risk_training.csv’ saved [694222/694222]



### Preview data

In [11]:
data_df = pd.read_csv("credit_risk_training.csv",
                    dtype={'LoanDuration': int, 'LoanAmount': int, 'InstallmentPercent': int, 'CurrentResidenceDuration': int, 'Age': int, 'ExistingCreditsCount': int, 'Dependents': int})

In [12]:
data_df

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk
5,no_checking,32,outstanding_credit,vacation,9604,500_to_1000,greater_7,6,male,co-applicant,...,unknown,57,none,free,2,skilled,2,yes,yes,Risk
6,no_checking,9,prior_payments_delayed,car_new,1032,100_to_500,4_to_7,3,male,none,...,savings_insurance,41,none,own,1,management_self-employed,1,none,yes,No Risk
7,less_0,16,credits_paid_to_date,vacation,3109,less_100,4_to_7,3,female,none,...,car_other,36,none,own,2,skilled,1,none,yes,No Risk
8,0_to_200,11,credits_paid_to_date,car_new,4553,less_100,less_1,3,female,none,...,savings_insurance,22,none,own,1,management_self-employed,1,none,yes,No Risk
9,no_checking,35,outstanding_credit,appliances,7138,500_to_1000,greater_7,5,male,co-applicant,...,unknown,49,none,free,2,skilled,2,yes,yes,Risk


# OpenScale configuration <a name="openscale"></a>

## DataMart setup

In [13]:
client.data_mart.setup(db_credentials=POSTGRES_CREDENTIALS, schema=SCHEMA_NAME)

In [14]:
data_mart_details = client.data_mart.get_details()

## Bind machine learning engines

### Bind  `CUSTOM` machine learning engine
**NOTE:** CUSTOM machine learning engine must follow this [API specification](https://aiopenscale-custom-deployement-spec.mybluemix.net/) to be supported.

Credentials support following fields:
- `url` - hostname and port (required)
- `request_headers` - headers
- `username` - part of BasicAuth (optional)
- `password` - part of BasicAuth (optional)

In [18]:
CUSTOM_ENGINE_CREDENTIALS = {
    "url": "***",
    "username": "***",
    "password": "***"
}

In [16]:
binding_uid = client.data_mart.bindings.add('My custom engine on Azure', CustomMachineLearningInstance(CUSTOM_ENGINE_CREDENTIALS))

In [17]:
bindings_details = client.data_mart.bindings.get_details()

In [18]:
client.data_mart.bindings.list()

a7d19c7c-30f7-4e4e-a8e1-41adee217c00,My custom engine on Azure,custom_machine_learning,2019-05-16T14:29:38.907Z


## Subscriptions

### Add subscriptions

List available deployments.

In [19]:
client.data_mart.bindings.list_assets()

credit,credit,2019-01-01T10:11:12Z,model,,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,False


Let's specify training data information like: list of features and list of categorical features required by fairness and explain.

In [20]:
feature_columns = ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker']
categorical_columns = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans','Housing', 'Job', 'Telephone', 'ForeignWorker']

In [21]:
subscription = client.data_mart.subscriptions.add(
    CustomMachineLearningAsset(
                source_uid='credit',
                label_column='Risk',
                prediction_column='Scored Labels',
                probability_column='Scored Probabilities',
                feature_columns=feature_columns.copy(),
                categorical_columns=categorical_columns.copy(),
                binding_uid=binding_uid))

### Get subscriptions list

In [22]:
subscriptions = client.data_mart.subscriptions.get_details()

In [23]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

['credit']


### List subscriptions

In [24]:
client.data_mart.subscriptions.list()

credit,credit,model,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,2019-05-16T14:30:14.730Z


# Test subscription by making scoring request and storing it in payload logging table

## Score the credit model

In [25]:
import requests
import time

values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 25, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
             4, "savings_insurance", 60, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
            ["0_to_200", 26, "all_credits_paid_back", "car_new", 863, "less_100", "less_1", 2, "female", "co-applicant",
             2, "real_estate", 38, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["0_to_200", 14, "no_credits", "car_new", 2368, "less_100", "1_to_4", 3, "female", "none", 3, "real_estate",
             29, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["0_to_200", 4, "no_credits", "car_new", 250, "less_100", "unemployed", 2, "female", "none", 3,
             "real_estate", 23, "none", "rent", 1, "management_self-employed", 1, "none", "yes"],
            ["no_checking", 17, "credits_paid_to_date", "car_new", 832, "100_to_500", "1_to_4", 2, "male", "none", 2,
             "real_estate", 42, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 50, "outstanding_credit", "appliances", 5696, "unknown", "greater_7", 4, "female",
             "co-applicant", 4, "unknown", 54, "none", "free", 2, "skilled", 1, "yes", "yes"],
            ["0_to_200", 13, "prior_payments_delayed", "retraining", 1375, "100_to_500", "4_to_7", 3, "male", "none", 3,
             "real_estate", 70, "none", "own", 2, "management_self-employed", 1, "none", "yes"]
        ]


request_data = {'fields': feature_columns, 'values': values}

header = {'Content-Type': 'application/json'}
scoring_url = subscription.get_details()['entity']['deployments'][0]['scoring_endpoint']['url']

start_time = time.time()
response = requests.post(scoring_url, json=request_data, headers=header)
response_time = int((time.time() - start_time)*1000)

response_data = response.json()
print('Response: ' + str(response_data))

Response: {'values': [['No Risk', [0.8823126094462725, 0.1176873905537274]], ['No Risk', [0.6755090846150376, 0.3244909153849625]], ['No Risk', [0.8944991421537971, 0.10550085784620292]], ['No Risk', [0.9297263621482206, 0.07027363785177945]], ['No Risk', [0.937346474163384, 0.06265352583661594]], ['No Risk', [0.8389265131291409, 0.16107348687085907]], ['Risk', [0.16270903114445467, 0.8372909688555453]], ['No Risk', [0.8011704003481404, 0.1988295996518596]]], 'fields': ['Scored Labels', 'Scored Probabilities']}


## Test payload logging

#### Using Python SDK

In [26]:
records_list = [PayloadRecord(request=request_data, response=response_data, response_time=response_time)]
subscription.payload_logging.store(records=records_list)

In [27]:
subscription.payload_logging.show_table()

be384d99-6476-4eb0-bd68-0044b19cedd2-1,2019-05-16 14:35:06.215000+00:00,credit,None,no_checking,13,credits_paid_to_date,car_new,1343,100_to_500,1_to_4,2,female,none,3,savings_insurance,25,none,own,2,skilled,1,none,yes,No Risk,"[0.8823126094462725, 0.1176873905537274]",0.8823126094462725
be384d99-6476-4eb0-bd68-0044b19cedd2-2,2019-05-16 14:35:06.215000+00:00,credit,None,no_checking,24,prior_payments_delayed,furniture,4567,500_to_1000,1_to_4,4,male,none,4,savings_insurance,60,none,free,2,management_self-employed,1,none,yes,No Risk,"[0.6755090846150376, 0.3244909153849625]",0.6755090846150376
be384d99-6476-4eb0-bd68-0044b19cedd2-3,2019-05-16 14:35:06.215000+00:00,credit,None,0_to_200,26,all_credits_paid_back,car_new,863,less_100,less_1,2,female,co-applicant,2,real_estate,38,none,own,1,skilled,1,none,yes,No Risk,"[0.8944991421537971, 0.10550085784620292]",0.8944991421537971
be384d99-6476-4eb0-bd68-0044b19cedd2-4,2019-05-16 14:35:06.215000+00:00,credit,None,0_to_200,14,no_credits,car_new,2368,less_100,1_to_4,3,female,none,3,real_estate,29,none,own,1,skilled,1,none,yes,No Risk,"[0.9297263621482206, 0.07027363785177945]",0.9297263621482206
be384d99-6476-4eb0-bd68-0044b19cedd2-5,2019-05-16 14:35:06.215000+00:00,credit,None,0_to_200,4,no_credits,car_new,250,less_100,unemployed,2,female,none,3,real_estate,23,none,rent,1,management_self-employed,1,none,yes,No Risk,"[0.937346474163384, 0.06265352583661594]",0.937346474163384
be384d99-6476-4eb0-bd68-0044b19cedd2-6,2019-05-16 14:35:06.215000+00:00,credit,None,no_checking,17,credits_paid_to_date,car_new,832,100_to_500,1_to_4,2,male,none,2,real_estate,42,none,own,1,skilled,1,none,yes,No Risk,"[0.8389265131291409, 0.16107348687085907]",0.8389265131291409
be384d99-6476-4eb0-bd68-0044b19cedd2-7,2019-05-16 14:35:06.215000+00:00,credit,None,no_checking,50,outstanding_credit,appliances,5696,unknown,greater_7,4,female,co-applicant,4,unknown,54,none,free,2,skilled,1,yes,yes,Risk,"[0.16270903114445467, 0.8372909688555453]",0.8372909688555453
be384d99-6476-4eb0-bd68-0044b19cedd2-8,2019-05-16 14:35:06.215000+00:00,credit,None,0_to_200,13,prior_payments_delayed,retraining,1375,100_to_500,4_to_7,3,male,none,3,real_estate,70,none,own,2,management_self-employed,1,none,yes,No Risk,"[0.8011704003481404, 0.1988295996518596]",0.8011704003481404


# Feedback logging & quality (accuracy) monitoring <a id="quality"></a>

## Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [28]:
subscription.quality_monitoring.enable(threshold=0.8, min_records=10)

### Test feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

The feedback records can be send to feedback table using below code.

In [29]:
records = [
    ["no_checking","28","outstanding_credit","appliances","5990","500_to_1000","greater_7","5","male","co-applicant","3","car_other","55","none","free","2","skilled","2","yes","yes","Risk"],
    ["greater_200","22","all_credits_paid_back","car_used","3376","less_100","less_1","3","female","none","2","car_other","32","none","own","1","skilled","1","none","yes","No Risk"],
    ["no_checking","39","credits_paid_to_date","vacation","6434","unknown","greater_7","5","male","none","4","car_other","39","none","own","2","skilled","2","yes","yes","Risk"],
    ["0_to_200","20","credits_paid_to_date","furniture","2442","less_100","unemployed","3","female","none","1","real_estate","42","none","own","1","skilled","1","none","yes","No Risk"],
    ["greater_200","4","all_credits_paid_back","education","4206","less_100","unemployed","1","female","none","3","savings_insurance","27","none","own","1","management_self-employed","1","none","yes","No Risk"],
    ["greater_200","23","credits_paid_to_date","car_used","2963","greater_1000","greater_7","4","male","none","4","car_other","46","none","own","2","skilled","1","none","yes","Risk"],
    ["no_checking","31","prior_payments_delayed","vacation","2673","500_to_1000","1_to_4","3","male","none","2","real_estate","35","stores","rent","1","skilled","2","none","yes","Risk"],
    ["no_checking","37","prior_payments_delayed","other","6971","500_to_1000","1_to_4","3","male","none","3","savings_insurance","54","none","own","2","skilled","1","yes","yes","Risk"],
    ["0_to_200","39","prior_payments_delayed","appliances","5685","100_to_500","1_to_4","4","female","none","2","unknown","37","none","own","2","skilled","1","yes","yes","Risk"],
    ["no_checking","38","prior_payments_delayed","appliances","4990","500_to_1000","greater_7","4","male","none","4","car_other","50","bank","own","2","unemployed","2","yes","yes","Risk"]]

fields = feature_columns.copy()
fields.append('Risk')

subscription.feedback_logging.store(feedback_data=records, fields=fields)

## Test model quality monitor

By default, quality monitoring is run on hourly schedule. You can also trigger it on demand using below code.

In [30]:
run_details = subscription.quality_monitoring.run(background_mode=False)




 Waiting for end of quality monitoring run bc3b8d78-2d65-4fc6-bd18-b1263c2aca25 




initializing
completed

---------------------------
 Successfully finished run 
---------------------------




### Show the quality metrics

In [32]:
subscription.quality_monitoring.show_table()

2019-05-16 14:37:22.307000+00:00,true_positive_rate,4d333f2f-c5e8-4ccc-a190-b1b1db5b6b25,0.42857142857142855,,,model_type: recommended,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit
2019-05-16 14:37:22.307000+00:00,area_under_roc,4d333f2f-c5e8-4ccc-a190-b1b1db5b6b25,0.7142857142857143,0.8,,model_type: recommended,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit
2019-05-16 14:37:22.307000+00:00,precision,4d333f2f-c5e8-4ccc-a190-b1b1db5b6b25,1.0,,,model_type: recommended,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit
2019-05-16 14:37:22.307000+00:00,f1_measure,4d333f2f-c5e8-4ccc-a190-b1b1db5b6b25,0.6,,,model_type: recommended,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit
2019-05-16 14:37:22.307000+00:00,accuracy,4d333f2f-c5e8-4ccc-a190-b1b1db5b6b25,0.6,,,model_type: recommended,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit
2019-05-16 14:37:22.307000+00:00,log_loss,4d333f2f-c5e8-4ccc-a190-b1b1db5b6b25,0.6451208770769135,,,model_type: recommended,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit
2019-05-16 14:37:22.307000+00:00,false_positive_rate,4d333f2f-c5e8-4ccc-a190-b1b1db5b6b25,0.0,,,model_type: recommended,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit
2019-05-16 14:37:22.307000+00:00,area_under_pr,4d333f2f-c5e8-4ccc-a190-b1b1db5b6b25,0.9142857142857141,,,model_type: recommended,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit
2019-05-16 14:37:22.307000+00:00,recall,4d333f2f-c5e8-4ccc-a190-b1b1db5b6b25,0.42857142857142855,,,model_type: recommended,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit
2019-05-16 14:37:22.307000+00:00,true_positive_rate,a5fa7bee-e727-4f6e-b8b6-9d59b863932a,0.42857142857142855,,,model_type: original,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit


# Fairness monitoring and explanations configuration

## Enable and test fairness monitoring

In [33]:
from ibm_ai_openscale.supporting_classes import Feature

subscription.fairness_monitoring.enable(
            features=[
                Feature("Sex", majority=['male'], minority=['female'], threshold=0.95),
                Feature("Age", majority=[[26, 75]], minority=[[18, 25]], threshold=0.95)
            ],
            favourable_classes=['No Risk'],
            unfavourable_classes=['Risk'],
            min_records=4,
            training_data=data_df
        )

In [34]:
fairness_run = subscription.fairness_monitoring.run(background_mode=False)




 Counting bias for deployment_uid=credit 




RUNNING
FINISHED

---------------------------
 Successfully finished run 
---------------------------




### Check fairness run results

In [35]:
subscription.fairness_monitoring.show_table()

2019-05-16 14:40:18.086844+00:00,Sex,female,False,1.0,100.0,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit,credit,
2019-05-16 14:40:18.086844+00:00,Age,"[18, 25]",False,1.0,100.0,a7d19c7c-30f7-4e4e-a8e1-41adee217c00,credit,credit,credit,


## Explainability configuration and test

### Enable explainability

In [36]:
subscription.explainability.enable(training_data=data_df)

#### Get sample transaction_id from payload logging table (`scoring_id`)

In [37]:
transaction_id = subscription.payload_logging.get_table_content(limit=1)['scoring_id'].values[0]

print(transaction_id)

be384d99-6476-4eb0-bd68-0044b19cedd2-1


### Run explanation for sample `transaction_id`

In [38]:
explain_run = subscription.explainability.run(transaction_id=transaction_id, background_mode=False)
        




 Looking for explanation for be384d99-6476-4eb0-bd68-0044b19cedd2-1 




in_progress....
finished

---------------------------
 Successfully finished run 
---------------------------




In [39]:
explain_result = pd.DataFrame.from_dict(explain_run['entity']['predictions'][0]['explanation_features'])
explain_result.plot.barh(x='feature_name', y='weight', color='g', alpha=0.8);

---

## Congratulations!
## The OpenScale configuration and test is completed!

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.